In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'
install = pd.read_pickle(PATH + 'inter/tmp/inst.pkl')
# train = pd.read_csv(PATH + application_train.csv)

In [2]:
train = pd.read_csv(PATH + 'application_train.csv')
test =  pd.read_csv(PATH + 'application_test.csv')
df = pd.concat([train, test], axis = 0)
df = df[['SK_ID_CURR']]
print(df.shape)

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


(356255, 1)


In [3]:
from sklearn.linear_model import LinearRegression

def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join(gp_col + [col])})
    return group

def count(df, gp_col, col):
    group = df[gp_col+[col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index()

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index()

def linear(df, gp_col, x_col_name, y_col_name):
    r = []
    for i, x in df.groupby(gp_col):
        lg = LinearRegression()
        lg1 = LinearRegression()
        
        lg.fit(x[[x_col_name]], x[[y_col_name]])
        x_max = max(x[x_col_name])
        x_min = min(x[x_col_name])
        lg1.fit((x[[x_col_name]]-x_max)/(x_max-x_min+1), x[[y_col_name]])
        r.append([i, lg.coef_[0][0], lg1.coef_[0][0]])
    tmpdf = pd.DataFrame(r, columns=[gp_col, x_col_name+'_lg_'+y_col_name, x_col_name+'_normallg_'+y_col_name])
    return tmpdf

def feature_in_time_window(df, gp_col, col, func, add_fun=None, n=None, time_col=None):
    tmp = None
    _df = df.copy()
    if n is not None:
        _df = _df[_df[time_col] >= n]
    if add_fun is None:
        tmp = func_(df[_df[time_col] >= n], gp_col, col)
    else:
        tmp = func(_df[_df[time_col] >= n], gp_col, col, add_fun)
    columns = [str(abs(n))+'_'+x for x in set(tmp.columns) - set(gp_col)]
    tmp.columns = columns
    return tmp

def area_under_curve(df, gp_col, x, y):
    gp = df.copy()
    gp_max = gp.groupby(gp_col)[[x]].max().reset_index().rename(columns={x:'max'})
    gp_min = gp.groupby(gp_col)[[x]].min().reset_index().rename(columns={x:'min'})
    gp = gp.merge(gp_max)
    gp = gp.merge(gp_min)
    gp['normal_x'] = (gp[x]-gp['max']) / (gp['max']-gp['min']+1)
    
    group = gp.groupby(gp_col)
    gp['tmp'] = (group[y].shift(-1)+gp[y]) * (group[x].shift(-1)-gp[x]) / 2
    gp['tmp_normal'] = (group[y].shift(-1)+gp[y]) * (group['normal_x'].shift(-1)-gp['normal_x']) / 2
    return gp.groupby(gp_col).agg({'tmp':'sum', 'tmp_normal':'sum'}).reset_index().rename(columns={'tmp':x+'_area_'+y, 'tmp_normal':x+'_normalarea_'+y})

'done'

'done'

In [4]:
install['DIFF_DAYS'] = install['DAYS_ENTRY_PAYMENT'] - install['DAYS_INSTALMENT']

In [5]:
# For each curr_id:
# total/max/mean late amount; total/max/mean late days; total late amount/total amt; sum(late days * late pct) 
# total/max/mean early amount; total/max/mean early days; total early amount div total amt; sum(early days * early pct)

In [6]:
late_index = install['DIFF_DAYS'] > 0

install['LATE_DAYS'] = 0
install.loc[late_index, 'LATE_DAYS'] = install.loc[late_index, 'DIFF_DAYS']
# Late amt
install['LATE_AMT'] = 0
install.loc[late_index, 'LATE_AMT'] = install.loc[late_index, 'AMT_PAYMENT']

# Late pct
install['LATE_PCT'] = 0
install.loc[late_index, 'LATE_PCT'] = install.loc[late_index, 'LATE_AMT']/install.loc[late_index, 'new_AMT_INSTALMENT']

install['LATE_AREA'] = 0
install.loc[late_index, 'LATE_AREA'] = install.loc[late_index, 'LATE_PCT'] * install['LATE_DAYS']

In [ ]:
for i in ['EARLY_', 'LATE_']:
    install[i+'DAYS']

In [7]:
early_index = install['DIFF_DAYS'] <= 0 
install['EARLY_DAYS'] = 0
install.loc[early_index, 'EARLY_DAYS'] = abs(install.loc[early_index, 'DIFF_DAYS'])

# early amt
install['EARLY_AMT'] = 0
install.loc[early_index, 'EARLY_AMT'] = install.loc[early_index, 'AMT_PAYMENT']

# Late pct
install['EARLY_PCT'] = 0
install.loc[early_index, 'EARLY_PCT'] = install.loc[early_index, 'EARLY_AMT']/install.loc[early_index, 'new_AMT_INSTALMENT']

install['EARLY_AREA'] = 0
install.loc[early_index, 'EARLY_AREA'] = install.loc[early_index, 'EARLY_PCT'] * install.loc[early_index,'EARLY_DAYS']

In [8]:

install['EARLY_AREA']

0            7.000000
1            0.000000
2            0.000000
3            8.000000
4            0.000000
5           33.000000
6            3.000000
7           26.000000
8            0.000000
9           39.000000
10           4.000000
11           7.000000
12           0.000000
13          45.000000
14           0.000000
15           0.000000
16           3.000000
17          10.996836
18          13.000000
19           0.000000
20           4.000000
21           3.000000
22           5.000000
23           0.000000
24           5.000000
25          15.000000
26          11.000000
27          13.000000
28           0.000000
29           1.000000
              ...    
13605371     0.000000
13605372     0.000000
13605373     0.000000
13605374     0.000000
13605375     0.000000
13605376     0.000000
13605377     0.000000
13605378     0.000000
13605379     0.000000
13605380     0.000000
13605381     0.000000
13605382     0.000000
13605383     0.000000
13605384     0.000000
13605385  

In [9]:
#total/max/mean late amount;
def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index()
# total/max/mean late days
df = df.merge( numerical(install, 'SK_ID_CURR', 'LATE_AMT', {'sum','mean','max'}), on = 'SK_ID_CURR', how = 'left')
df = df.merge( numerical(install, 'SK_ID_CURR', 'LATE_DAYS', {'mean','max'}), on = 'SK_ID_CURR', how = 'left')
df = df.merge( numerical(install, 'SK_ID_CURR', 'LATE_AREA', {'sum'}), on = 'SK_ID_CURR', how = 'left')

df = df.merge( numerical(install, 'SK_ID_CURR', 'EARLY_AMT', {'sum','mean','max'}), on = 'SK_ID_CURR', how = 'left')
df = df.merge( numerical(install, 'SK_ID_CURR', 'EARLY_DAYS', {'mean','max'}), on = 'SK_ID_CURR', how = 'left')
df = df.merge( numerical(install, 'SK_ID_CURR', 'EARLY_AREA', {'sum'}), on = 'SK_ID_CURR', how = 'left')


In [10]:
df.head(20)

,SK_ID_CURR,LATE_AMT_mean,LATE_AMT_sum,LATE_AMT_max,LATE_DAYS_mean,LATE_DAYS_max,LATE_AREA_sum,EARLY_AMT_mean,EARLY_AMT_sum,EARLY_AMT_max,EARLY_DAYS_mean,EARLY_DAYS_max,EARLY_AREA_sum
0,100002,0.000000,0.00,0.000,0.000000,0.0,0.000000,11559.247105,219625.695,53093.745,20.421053,31.0,388.000000
1,100003,0.000000,0.00,0.000,0.000000,0.0,0.000000,64754.586000,1618864.650,560835.360,7.160000,14.0,179.000000
2,100004,0.000000,0.00,0.000,0.000000,0.0,0.000000,7096.155000,21288.465,10573.965,7.666667,11.0,23.000000
3,100006,0.000000,0.00,0.000,0.000000,0.0,0.000000,62947.088438,1007153.415,691786.890,19.375000,77.0,310.000000
4,100007,1182.470455,78043.05,22678.785,0.954545,12.0,62.991512,11031.589773,728084.925,22678.785,4.590909,31.0,212.059995
5,100008,6.490286,227.16,227.160,37.628571,1317.0,24.959607,27354.012429,957390.435,432218.295,11.514286,28.0,402.488300
6,100009,120.691765,6155.28,6155.280,0.137255,7.0,7.000000,9447.840000,481839.840,17341.605,8.725490,23.0,445.000000
7,100010,0.000000,0.00,0.000,0.000000,0.0,0.000000,27449.208000,274492.080,27463.410,11.900000,30.0,119.000000
8,100011,2803.246615,364422.06,31295.250,7.946154,958.0,79.909951,8525.647038,1108334.115,46485.000,4.807692,32.0,406.388848
9,100012,374.152500,17959.32,11057.535,0.562500,14.0,12.304776,10077.133125,483702.390,58687.245,25.875000,90.0,1208.183847


In [12]:
print(train.shape, test.shape)
df.to_pickle(PATH +'test/install2curr.pkl')


(307511, 122) (48744, 121)


In [13]:
df

,SK_ID_CURR,LATE_AMT_mean,LATE_AMT_sum,LATE_AMT_max,LATE_DAYS_mean,LATE_DAYS_max,LATE_AREA_sum,EARLY_AMT_mean,EARLY_AMT_sum,EARLY_AMT_max,EARLY_DAYS_mean,EARLY_DAYS_max,EARLY_AREA_sum
0,100002,0.000000,0.000,0.000,0.000000,0.0,0.000000,11559.247105,219625.695,53093.745,20.421053,31.0,388.000000
1,100003,0.000000,0.000,0.000,0.000000,0.0,0.000000,64754.586000,1618864.650,560835.360,7.160000,14.0,179.000000
2,100004,0.000000,0.000,0.000,0.000000,0.0,0.000000,7096.155000,21288.465,10573.965,7.666667,11.0,23.000000
3,100006,0.000000,0.000,0.000,0.000000,0.0,0.000000,62947.088438,1007153.415,691786.890,19.375000,77.0,310.000000
4,100007,1182.470455,78043.050,22678.785,0.954545,12.0,62.991512,11031.589773,728084.925,22678.785,4.590909,31.0,212.059995
5,100008,6.490286,227.160,227.160,37.628571,1317.0,24.959607,27354.012429,957390.435,432218.295,11.514286,28.0,402.488300
6,100009,120.691765,6155.280,6155.280,0.137255,7.0,7.000000,9447.840000,481839.840,17341.605,8.725490,23.0,445.000000
7,100010,0.000000,0.000,0.000,0.000000,0.0,0.000000,27449.208000,274492.080,27463.410,11.900000,30.0,119.000000
8,100011,2803.246615,364422.060,31295.250,7.946154,958.0,79.909951,8525.647038,1108334.115,46485.000,4.807692,32.0,406.388848
9,100012,374.152500,17959.320,11057.535,0.562500,14.0,12.304776,10077.133125,483702.390,58687.245,25.875000,90.0,1208.183847
